In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
training_size = 50000
testing_size = 10000

In [ ]:
training_dataset = unpickle('images/cifar-100-python/train')
testing_dataset = unpickle('images/cifar-100-python/test')

In [ ]:
training_images = training_dataset[b'data']
training_labels = training_dataset[b'fine_labels']
testing_images = testing_dataset[b'data']
testing_labels = testing_dataset[b'fine_labels']

In [ ]:
training_images = training_images[:training_size]
training_labels = training_labels[:training_size]
testing_images = testing_images[:testing_size]
testing_labels = testing_labels[:testing_size]

In [ ]:
training_images = training_images.reshape(len(training_images),3,32,32).transpose(0,2,3,1)
training_labels = np.array(training_labels)
training_labels = training_labels.reshape(len(training_images), 1)

testing_images = testing_images.reshape(len(testing_images),3,32,32).transpose(0,2,3,1)
testing_labels = np.array(testing_labels)
testing_labels = testing_labels.reshape(len(testing_images), 1)

In [ ]:
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [ ]:
h = []
h_test = []

In [ ]:
hog = cv2.HOGDescriptor((32,32), (16,16), (8,8), (8,8),9)

In [ ]:
for x in training_images:
    h.append(hog.compute(x))

for x in testing_images:
    h_test.append(hog.compute(x))

print("hog computed")

hog computed


In [ ]:
h = np.array(h)
h_test = np.array(h_test)
print(h.shape)
print(h_test.shape)

(50000, 324)
(10000, 324)


In [ ]:
x_train = h
y_train = training_labels

x_test = h_test
y_test = testing_labels

In [ ]:
y_train=y_train.reshape(-1,)
y_test=y_test.reshape(-1,)

In [ ]:
def kMeans_init_centroids(X, K):
    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]
    return centroids

In [ ]:
def cost_kmeans(X,Y,centroids):
    total_cost=np.zeros_like(centorids)    
    clusters=0
    for d in range(centroids.shape[0]):
        cost=0
        m =0
        for i in range(X.shape[0]):
            if Y[i]==(clusters):
                m+=1
                cost_i=((X[i]-centroids[clusters])**2)
                cost+=cost_i
        total_cost[clusters]=(1/m *(cost))
        clusters+=1
    return total_cost

In [ ]:
def mean_kmeans(X,Y,centroids):
    clusters=0
    for d in range(centroids.shape[0]):
        points=np.zeros_like(X[1])
        m =0
        for i in range(X.shape[0]):
            if Y[i]==clusters:
                m+=1
                points+=X[i]
        centroids[clusters]=(1/m)*(points)
        clusters+=1
    return centroids

In [ ]:
k=100 
centorids =kMeans_init_centroids(h,k)
print(centorids)
print("\n\n")
P_cost=np.zeros_like(centorids)
cost=cost_kmeans(h,y_train,centorids)
centorids=mean_kmeans(h,y_train,centorids)
print(centorids)

[[0.03013249 0.09902567 0.26710266 ... 0.00399128 0.01101066 0.0197227 ]
 [0.16334087 0.23737893 0.21100785 ... 0.15301411 0.20694393 0.22013374]
 [0.05333689 0.12972106 0.27701008 ... 0.01860613 0.026985   0.03468471]
 ...
 [0.06133282 0.2535168  0.2535168  ... 0.09027507 0.13191234 0.14579685]
 [0.01650241 0.02453853 0.08495563 ... 0.13547303 0.05482215 0.12271678]
 [0.09142247 0.03798105 0.07037906 ... 0.10829569 0.05800415 0.08798826]]



[[0.11711401 0.19511959 0.21482125 ... 0.059437   0.05981212 0.08955865]
 [0.11417328 0.11843795 0.12980707 ... 0.11986747 0.10996486 0.12892663]
 [0.14238523 0.13579291 0.12878266 ... 0.14137585 0.14341445 0.15619946]
 ...
 [0.1632033  0.13707139 0.11674014 ... 0.12205484 0.13432808 0.17372546]
 [0.16031691 0.15100414 0.12348669 ... 0.12536335 0.13659522 0.17040034]
 [0.1119085  0.12933935 0.13473973 ... 0.09916387 0.09953047 0.11502664]]


In [ ]:
def compute_centroids(X, idx, K):
    m, n = X.shape
    centroids = np.zeros((K, n))
    for i  in range(K):
        k_pi=X[idx == i]
        centroids[i] = np.mean(k_pi, axis = 0)
    return centroids

In [ ]:
def Kmean_acc(idx,y_train):
    correct=0
    for i in range(50000):
        if (idx[i]-y_train[i])==0:
            correct = correct + 1
    acc = (correct/50000)*100
    print('Accuracy:',acc,'%')

In [ ]:
def find_closest_centroids(X, centroids):
    K = centroids.shape[0]
    idx = np.zeros(X.shape[0], dtype=int)
    for i in range(X.shape[0]) :
        dist=[]
        for j in range(centroids.shape[0]):
            norm=  np.linalg.norm(X[i] - centroids[j])  
            dist.append(norm)
        idx[i]=np.argmin(dist)
    return idx

In [ ]:
def run_kMeans(X, initial_centroids, max_iters=10):
    m, n = X.shape
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    previous_centroids = centroids    
    idx = np.zeros(m)
    for i in range(max_iters):
        print("K-Means iteration %d/%d" % (i, max_iters-1))
        idx = find_closest_centroids(X, centroids)
        centroids = compute_centroids(X, idx, K)
    return centroids, idx

In [ ]:
centroids, idx = run_kMeans(x_train,centorids,max_iters=1)

K-Means iteration 0/0


In [ ]:
print (idx)
print (y_train)

[77 29 58 ...  3 30 58]
[19 29  0 ...  3  7 73]


In [ ]:
Kmean_acc(idx,y_train)

Accuracy: 16.342000000000002 %
